# Museums in the Pandemic - Social media analytics

**Authors**: Andrea Ballatore (KCL)

**Abstract**: Analysis of social media data

## Setup
This is to check that your environment is set up correctly (it should print 'env ok', ignore warnings).

In [2]:
# Test geospatial libraries
# check environment
import os
print("Conda env:", os.environ['CONDA_DEFAULT_ENV'])
if os.environ['CONDA_DEFAULT_ENV'] != 'mip_v1':
    raise Exception("Set the environment 'mip_v1' on Anaconda. Current environment: " + os.environ['CONDA_DEFAULT_ENV'])

# spatial libraries 
import pandas as pd
import pickle
import spacy
import itertools
import re
from termcolor import colored
import matplotlib.dates as mdates
import sys
from matplotlib.colors import ListedColormap
import numpy as np
import calplot
from numpy import arange
#import tensorflow as tf
from bs4 import BeautifulSoup
from bs4.element import Comment
#import torch
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

# import from `mip` project
print(os.getcwd())
fpath = os.path.abspath('../')
if not fpath in sys.path:
    sys.path.insert(0, fpath)

out_folder = '../../'

from museums import *
from utils import _is_number
from analytics.text_models import derive_new_attributes_matches, get_all_matches_from_db, get_indicator_annotations
from museums import get_museums_w_web_urls, get_twitter_facebook_links, get_extra_museum_attributes

cur_folder = out_folder + 'data/analysis/'

print('env ok')

Conda env: mip_v1
/Users/andreaballatore/Dropbox/DRBX_Docs/Work/Projects/github_projects/museums-in-the-pandemic/mip/notebooks_py
env ok


## Connect to DB

It needs the DCS VPN active to work.

In [2]:
# open connection to DB
from db.db import connect_to_postgresql_db

db_conn = connect_to_postgresql_db()
print("DB connected")

DB connected


## Load museum info


In [7]:
# load museums
df = get_museums_w_web_urls(out_folder)
print("museums url N:",len(df))
attr_df = load_input_museums_wattributes(out_folder)
df = pd.merge(df, attr_df, on='muse_id', how='left')
print("museum df with attributes: len", len(df))
mus_df = df.rename(columns={'muse_id':'museum_id','musname_x':'musname'})
del df
print(len(mus_df), mus_df.columns)

links_df = get_twitter_facebook_links(out_folder)[['museum_id', 'twitter_id', 'facebook_pages']]
mus_df = mus_df.merge(links_df, on='museum_id', how='left')
mus_df = get_extra_museum_attributes(mus_df)
del links_df
#print(len(mus_df), mus_df.columns)
mus_df.sample(50)

museums urls: ../../data/museums/museum_websites_urls-v3.tsv
nationaltrust.org.uk       179
english-heritage.org.uk     52
no_resource.                33
visitscotland.com           24
nts.org.uk                  21
                          ... 
glynnvivian.co.uk            1
glynde.co.uk                 1
gwsr.com                     1
gloucesterquays.co.uk        1
smithsonfarm.co.uk           1
Name: domain, Length: 2441, dtype: int64
get_museums_w_web_urls Museums=3344 URLs=3344
museums url N: 3344
Index(['musname', 'muse_id', 'town', 'postcode', 'accreditation', 'governance',
       'size', 'subject_matter', 'closing_date', 'provenance',
       'deprivation_index', 'geodemographic_group', 'geodemographic_subgroup',
       'admin_area'],
      dtype='object')
loaded museums w attributes (open): 3341 data/museums/museums_wattributes-2020-02-23.tsv
museum df with attributes: len 3344
3344 Index(['museum_id', 'musname', 'town_x', 'url', 'url_source', 'Unnamed: 5',
       'domain', 'm

/Users/andreaballatore/Dropbox/DRBX_Docs/Work/Projects/github_projects/museums-in-the-pandemic/mip/museums.py:401: FutureWarning: The default value of regex will change from True to False in a future version.
  df['region'] = df['region'].str.replace('\(English Region\)','')


,museum_id,musname,town_x,url,url_source,Unnamed: 5,domain,musname_y,town_y,postcode,...,deprivation_index,geodemographic_group,geodemographic_subgroup,admin_area,twitter_id,facebook_pages,governance_simpl,subject_matter_simpl,country,region
3112,mm.domus.NE072,Washington Old Hall,Washington Village,https://www.nationaltrust.org.uk/washington-ol...,pred,NaN,nationaltrust.org.uk,Washington Old Hall,Washington Village,NE38 7LE,...,7.0,Services Manufacturing and Mining Legacy,Mining Legacy,/England/North East (English Region)/North Eas...,twitter.com/nt_woh,"['www.facebook.com/washingtonoldhall', 'en-gb....",independent,Buildings,England,North East
3060,mm.musa.016,"University of the Arts, London Galleries",Seaford,https://www.arts.ac.uk/about-ual/collections-a...,pred,NaN,arts.ac.uk,"University of the Arts, London Galleries",Seaford,SE1 6SB,...,2.0,London Cosmopolitan,London Cosmopolitan,/England/London (English Region)/Southwark (Lo...,"['twitter.com/ual', 'twitter.com/portscitymuse...",['www.facebook.com/universityoftheartslondon'],university,Arts,England,London
371,mm.domus.SW151,Bridport Museum,Bridport,https://www.bridportmuseum.co.uk/,manual,NaN,bridportmuseum.co.uk,Bridport Museum,Bridport,DT6 3NR,...,5.0,English and Welsh Countryside,Older Farming Communities,/England/South West (English Region)/Dorset (E...,twitter.com/bridportmuseum,"['www.facebook.com/bridportmuseum', 'en-gb.fac...",government,Local Histories,England,South West
1495,mm.aim.0528,Land of Lost Content,Craven Arms,https://poets.org/poem/shropshire-lad-xl,pred,NaN,poets.org,Land of Lost Content,Craven Arms,SY7 9NW,...,5.0,English and Welsh Countryside,Sparse English and Welsh Countryside,/England/West Midlands (English Region)/Shrops...,['twitter.com/poetsorg'],"['www.facebook.com/poets.org', 'en-gb.facebook...",independent,Other,England,West Midlands
3183,mm.domus.SE403,White Mill Rural Heritage Centre,Sandwich,http://whitemillheritagecentre.org.uk/site/,pred,NaN,whitemillheritagecentre.org.uk,White Mill Rural Heritage Centre,Sandwich,CT13 9JB,...,6.0,Remoter Coastal Living,Seaside Living,/England/South East (English Region)/Kent (Eng...,twitter.com/whitemillmuseum,['www.facebook.com/whitemillruralheritagecentre'],independent,Rural Industry,England,South East
2390,mm.domus.NW136,Salford Museum And Art Gallery,Salford,https://salfordmuseum.com/,pred,NaN,salfordmuseum.com,Salford Museum And Art Gallery,Salford,M5 4WU,...,2.0,Manufacturing Traits,Industrial and Multi-ethnic,/England/North West (English Region)/Greater M...,twitter.com/salfordmuseum,"['www.facebook.com/salfordmuseum', 'en-gb.face...",government,Mixed,England,North West
1726,mm.domus.SE600,Milestones - Hampshires Living History Museum,Basingstoke,https://www.milestonesmuseum.org.uk/,pred,NaN,milestonesmuseum.org.uk,Milestones - Hampshires Living History Museum,Basingstoke,RG22 6PG,...,6.0,Thriving Rural,Rural Growth Areas,/England/South East (English Region)/Hampshire...,twitter.com/milestoneshct,"['en-gb.facebook.com/milestonesmuseum', 'www.f...",government,Local Histories,England,South East
3102,mm.domus.SW145,Wareham Town Museum,Wareham,http://www.greenacre.info/WTM/,pred,NaN,greenacre.info,Wareham Town Museum,Wareham,BH20 4NN,...,4.0,English and Welsh Countryside,Older Farming Communities,/England/South West (English Region)/Dorset (E...,no_resource,en-gb.facebook.com/pages/category/History-Muse...,government,Local Histories,England,South West
2745,mm.aim.1006,The Bugatti Trust Museum,Cheltenham,http://www.bugatti-trust.co.uk/,pred,NaN,bugatti-trust.co.uk,The Bugatti Trust Museum,Cheltenham,GL52 9RD,...,10.0,Country Living,Country Living,/England/South West (English Region)/Glouceste...,twitter.com/bugattitrust,['www.facebook.com/groups/889331591127366'],independent,Transport,England,South West
2236,mm.domus.SE092,Ramsey Rural Museum,Huntingdon,https://ramseyruralmuseum.co.uk/,pred,NaN,ramseyruralmuseum.co.uk,Ramsey Rural Museum,Huntingdon,PE26 1UY,...,6.0,Country Living,Country Living,/En

## Temporal analysis of Twitter/Facebook

Temporal analysis based on message counts.

In [4]:
# file paths
tw_fn = out_folder+'tmp/tweets.pik'
fb_fn = out_folder+'tmp/facebook_posts.pik'

### Download Twitter/Facebook data from DB (slow)

In [5]:
if False:
    sql = "select muse_id as museum_id, account, tw_ts as ts from twitter.tweets_dump td;" # limit 10000
    df = pd.read_sql(sql, db_conn)
    df['ts'] = pd.to_datetime(df['ts'])
    print(df.columns)
    df = df.set_index('ts')
    df.to_pickle(tw_fn)
    print(len(df))
    tw_fn

    sql = "select museum_id, post_ts as ts from facebook.facebook_posts_dump;" # limit 10000
    df = pd.read_sql(sql, db_conn)
    df['post_ts'] = pd.to_datetime(df['post_ts'])
    print(df.columns)
    df = df.set_index('post_ts')
    df.to_pickle(fb_fn)
    print(len(df))
    fb_fn

### Activity by museum

In [8]:
def messages_by_museum(df, label):
    print('messages_by_museum', label, len(df))
    mdf = df.groupby('museum_id').size()
    field = 'msg_count_'+label
    mdf = mdf.reset_index(name=field)
    mdf[field+'_decile'] = pd.qcut(-mdf[field], 10, labels=False)+1
    mdf[field+'_z'] = round((mdf[field] - mdf[field].mean())/mdf[field].std(),3)
    print(mdf.columns)
    return mdf
    
# generate tweet plots
df = pd.read_pickle(tw_fn)
act_df = messages_by_museum(df, 'twitter')

df = pd.read_pickle(fb_fn)
act_df2 = messages_by_museum(df, 'facebook')
act_df = act_df.merge(act_df2, on='museum_id', how='outer')

# merge with all museums
act_df = act_df.merge(mus_df, on='museum_id', how='outer')
act_df['msg_count_facebook'] = act_df['msg_count_facebook'].fillna(0)
act_df['msg_count_twitter'] = act_df['msg_count_twitter'].fillna(0)

fout = cur_folder + 'temporal_analysis/msg_count_by_museum.xlsx'
act_df.to_excel(fout, index=False)
fout

messages_by_museum twitter 5653012
Index(['museum_id', 'msg_count_twitter', 'msg_count_twitter_decile',
       'msg_count_twitter_z'],
      dtype='object')
messages_by_museum facebook 1468515
Index(['museum_id', 'msg_count_facebook', 'msg_count_facebook_decile',
       'msg_count_facebook_z'],
      dtype='object')


'../../data/analysis/temporal_analysis/msg_count_by_museum.xlsx'

#### Sample for Tw/Fb validation

Validation carried out on 2021 Nov 24.
Stratified by decile to validate the low deciles.

In [40]:
# msg_count_twitter_decile | msg_count_facebook_decile
dfs = []
for dec_var in ['msg_count_twitter_decile','msg_count_facebook_decile']:
    for dec, subdf in act_df.groupby(dec_var):
        if dec < 3: continue
        print(dec, len(subdf))
        smpl_df = subdf.sample(10, random_state=10)
        smpl_df['sample'] = "{}_sample_{}".format(dec_var,dec)
        dfs.append(smpl_df)
decile_sample_df = pd.concat(dfs)
print(decile_sample_df.columns)
del dfs, subdf

print("N =",len(decile_sample_df))
decile_sample_df = decile_sample_df[['museum_id','musname','town_x','url','twitter_id','facebook_pages',
                  'msg_count_twitter','msg_count_facebook','sample']]
decile_sample_df['correct'] = ''
decile_sample_df.to_excel(out_folder+'tmp/museum_social_media_sample_by_decile.xlsx',index=False)

3.0 220
4.0 220
5.0 219
6.0 220
7.0 222
8.0 218
9.0 228
10.0 213
3.0 125
4.0 125
5.0 127
6.0 123
7.0 126
8.0 124
9.0 125
10.0 125
Index(['museum_id', 'msg_count_twitter', 'msg_count_twitter_decile',
       'msg_count_twitter_z', 'msg_count_facebook',
       'msg_count_facebook_decile', 'msg_count_facebook_z', 'musname',
       'town_x', 'url', 'url_source', 'Unnamed: 5', 'domain', 'musname_y',
       'town_y', 'postcode', 'accreditation', 'governance', 'size',
       'subject_matter', 'closing_date', 'provenance', 'deprivation_index',
       'geodemographic_group', 'geodemographic_subgroup', 'admin_area',
       'twitter_id', 'facebook_pages', 'governance_simpl',
       'subject_matter_simpl', 'country', 'region', 'sample'],
      dtype='object')
N = 160


#### Total stats

In [5]:
# activity stats
for plat in ['twitter','facebook']:
    df = act_df.copy()
    df['msg_count_'+plat] = df['msg_count_'+plat] + 1
    ax = sns.histplot(data=df, x="msg_count_"+plat, kde=True, log_scale=True)
    ax.set_title('Museums on '+plat)
    ax.set_xlabel('Number of messages (total)')
    ax.set_ylabel('N of museums')
    fout = cur_folder+'/temporal_analysis/museum_activity_hist_{}.pdf'.format(plat)
    plt.savefig(fout)
    plt.clf()
    print(fout)
    del df

NameError: name 'act_df' is not defined

#### Calc and viz group stats

In [78]:
# governance / size stats
# attr

for x in [cur_folder+'temporal_analysis/pivot_tables/']:
    print(x)
    if not os.path.exists(x):
        os.makedirs(x)

def q25(x): return x.quantile(0.25)

def q75(x): return x.quantile(0.75)

def active_mus_n(x): return len(x[x>0])

def active_mus_pc(x):
    # dummy value
    return -1

def active_mus_pc_z(x):
    # dummy value
    return -1

def msg_per_mus(x):
    # dummy value
    return -1

def make_multilevel_df_flat(df):
    vdf = df.reset_index(level=[0,1])
    assert len(vdf.columns)==3
    # from long form to narrow form
    mdf = vdf.pivot(index=vdf.columns[0], columns=vdf.columns[1], values=vdf.columns[2])
    return mdf

def plot_bivar_heatmap(df, label, var, out_fold):
    print('plot_bivar_heatmap:', label, var)
    
    df = df.copy()    
    df.columns = ["_".join(a) for a in df.columns.to_flat_index()]
    cmap = "YlGnBu"
    #cmap = "YlOrBr"
    if 'pc_z' in var:
        cmap = "vlag_r"
    if '_twitter_count' in var or '_facebook_count' in var:
        cmap = "OrRd"
    assert var in df.columns, "{} not in {}".format(var, df.columns)
    df = df[var]
    df = make_multilevel_df_flat(df)
    
    # tune font sizes
    font_scale = .8
    font_sz = 8
    n_cells = len(df.columns) * len(df)
    if n_cells > 25:
        # small plots
        font_scale = .5
        font_sz = 3
    if n_cells > 50:
        # small plots
        font_scale = .3
        font_sz = 2
    sns.set(font_scale=font_scale)
    
    # plot missing values
    sns.heatmap(
        np.where(df.isna(), 0, np.nan),
        cbar=False,
        annot=np.full_like(df, "NA", dtype=object),
        fmt="", 
        annot_kws={"size": font_sz, "va": "center_baseline", "color": "black"},
        cmap=ListedColormap(['whitesmoke']),
        linewidth=0)
    
    # plot heatmap
    ax = sns.heatmap(df, annot=True, linewidth=2, square=True, annot_kws={"size": font_sz},
                     cmap=cmap, fmt='g', linecolor='white', cbar_kws={"shrink": .5})
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
    
    # fix for count name confusion
    var = var.replace('msg_count_twitter_count','museums_n')
    var = var.replace('msg_count_facebook_count','museums_n')
    
    ax.set_title("[{}] by [{}]".format(var, label))
    plt.tight_layout()
    fn = out_fold + 'heatmap_bivar__{}-{}.pdf'.format(label,var)
    plt.savefig(fn)
    plt.clf()
    sns.set(font_scale=1)
    print(fn)

def activity_stats_mus_groups(df, vars):
    print('\nactivity_stats_mus_groups', vars)
    #print(df.columns)
    if len(vars) == 2:
        if vars[0] in vars[1] or vars[1] in vars[0]: 
            return None
    stats_cols = ['sum','mean','std','min',q25,'median',q75,'max','count',
                  msg_per_mus,active_mus_n,active_mus_pc,active_mus_pc_z]
    stats_df = df.groupby(vars).agg({
        'msg_count_twitter': stats_cols,
        'msg_count_facebook': stats_cols
    }).round(1)
    # calculate derived fields
    for c in ['msg_count_twitter','msg_count_facebook']:
        stats_df[(c,'active_mus_pc')] = round(stats_df[(c,'active_mus_n')]/stats_df[(c,'count')]*100,1)
        stats_df[(c,'msg_per_mus')] = round(stats_df[(c,'sum')]/stats_df[(c,'active_mus_n')],1)
        x = stats_df[(c,'active_mus_pc')]
        stats_df[(c,'active_mus_pc_z')] = round((x - x.mean())/x.std(), 1)
        stats_df[(c,'active_mus_pc_z')]
    return stats_df

../../data/analysis/temporal_analysis/pivot_tables/


In [80]:
# generate plots
var_combinations = ['subject_matter_simpl','governance','governance_simpl','size','subject_matter',
        'accreditation','region','country']
var_combinations2 = []
for x1 in var_combinations:
    for x2 in var_combinations:
        if x1 != x2 and (not x2 in x1 and not x1 in x2):
            var_combinations2.append([min(x1,x2),max(x1,x2)])
for v in var_combinations:
    var_combinations2.append([v])
    
del var_combinations
#var_combinations = [['governance'],['governance','region'],['size','governance_simpl',['subject_matter_simpl']]] # DEBUG

# generate all combinations
for attrs in var_combinations2:
    # calc stats
    attrs = list(attrs)
    df = activity_stats_mus_groups(act_df, attrs)
    if df is None: continue
    attrs_str = '-'.join(attrs)
    fcomb_fold = cur_folder+'temporal_analysis/pivot_tables/'+attrs_str+'/'
    print(fcomb_fold)
    if not os.path.exists(fcomb_fold):
        os.makedirs(fcomb_fold)
    
    if len(attrs) == 2:
        # plot tables with heatmaps
        for plat in ['msg_count_facebook','msg_count_twitter']:
            for var in ['count','median','mean','q75','active_mus_pc','active_mus_pc_z','msg_per_mus']:
                target_var = plat + '_' + var
                plot_bivar_heatmap(df, attrs_str, target_var, fcomb_fold)
                
    fout = fcomb_fold+'museum_activity_groups__var{}-{}.xlsx'.format(len(attrs), attrs_str)
    df.to_excel(fout,index=True)
    
    # plot
    if False:
        for att in attrs:
            ax = sns.boxplot(x="msg_count_twitter", y=att, # hue="governance",
                         data=act_df, palette="Set3")
            ax.set_xscale("log")
            plt.show()
            plt.clf()


activity_stats_mus_groups ['governance', 'subject_matter_simpl']
../../data/analysis/temporal_analysis/pivot_tables/governance-subject_matter_simpl/
plot_bivar_heatmap: governance-subject_matter_simpl msg_count_facebook_count
../../data/analysis/temporal_analysis/pivot_tables/governance-subject_matter_simpl/heatmap_bivar__governance-subject_matter_simpl-museums_n.pdf
plot_bivar_heatmap: governance-subject_matter_simpl msg_count_facebook_median
../../data/analysis/temporal_analysis/pivot_tables/governance-subject_matter_simpl/heatmap_bivar__governance-subject_matter_simpl-msg_count_facebook_median.pdf
plot_bivar_heatmap: governance-subject_matter_simpl msg_count_facebook_mean
../../data/analysis/temporal_analysis/pivot_tables/governance-subject_matter_simpl/heatmap_bivar__governance-subject_matter_simpl-msg_count_facebook_mean.pdf
plot_bivar_heatmap: governance-subject_matter_simpl msg_count_facebook_q75
../../data/analysis/temporal_analysis/pivot_tables/governance-subject_matter_simpl

/Users/andreaballatore/opt/anaconda3/envs/mip_v1/lib/python3.8/site-packages/seaborn/matrix.py:198: RuntimeWarning: All-NaN slice encountered
  vmin = np.nanmin(calc_data)
/Users/andreaballatore/opt/anaconda3/envs/mip_v1/lib/python3.8/site-packages/seaborn/matrix.py:203: RuntimeWarning: All-NaN slice encountered
  vmax = np.nanmax(calc_data)


../../data/analysis/temporal_analysis/pivot_tables/accreditation-subject_matter_simpl/heatmap_bivar__accreditation-subject_matter_simpl-museums_n.pdf
plot_bivar_heatmap: accreditation-subject_matter_simpl msg_count_facebook_median


/Users/andreaballatore/opt/anaconda3/envs/mip_v1/lib/python3.8/site-packages/seaborn/matrix.py:198: RuntimeWarning: All-NaN slice encountered
  vmin = np.nanmin(calc_data)
/Users/andreaballatore/opt/anaconda3/envs/mip_v1/lib/python3.8/site-packages/seaborn/matrix.py:203: RuntimeWarning: All-NaN slice encountered
  vmax = np.nanmax(calc_data)


../../data/analysis/temporal_analysis/pivot_tables/accreditation-subject_matter_simpl/heatmap_bivar__accreditation-subject_matter_simpl-msg_count_facebook_median.pdf
plot_bivar_heatmap: accreditation-subject_matter_simpl msg_count_facebook_mean


...

/Users/andreaballatore/opt/anaconda3/envs/mip_v1/lib/python3.8/site-packages/seaborn/matrix.py:198: RuntimeWarning: All-NaN slice encountered
  vmin = np.nanmin(calc_data)
/Users/andreaballatore/opt/anaconda3/envs/mip_v1/lib/python3.8/site-packages/seaborn/matrix.py:203: RuntimeWarning: All-NaN slice encountered
  vmax = np.nanmax(calc_data)


../../data/analysis/temporal_analysis/pivot_tables/accreditation-size/heatmap_bivar__accreditation-size-msg_count_twitter_active_mus_pc_z.pdf
plot_bivar_heatmap: accreditation-size msg_count_twitter_msg_per_mus


/Users/andreaballatore/opt/anaconda3/envs/mip_v1/lib/python3.8/site-packages/seaborn/matrix.py:198: RuntimeWarning: All-NaN slice encountered
  vmin = np.nanmin(calc_data)
/Users/andreaballatore/opt/anaconda3/envs/mip_v1/lib/python3.8/site-packages/seaborn/matrix.py:203: RuntimeWarning: All-NaN slice encountered
  vmax = np.nanmax(calc_data)


../../data/analysis/temporal_analysis/pivot_tables/accreditation-size/heatmap_bivar__accreditation-size-msg_count_twitter_msg_per_mus.pdf

activity_stats_mus_groups ['accreditation', 'subject_matter']
../../data/analysis/temporal_analysis/pivot_tables/accreditation-subject_matter/
plot_bivar_heatmap: accreditation-subject_matter msg_count_facebook_count
../../data/analysis/temporal_analysis/pivot_tables/accreditation-subject_matter/heatmap_bivar__accreditation-subject_matter-museums_n.pdf
plot_bivar_heatmap: accreditation-subject_matter msg_count_facebook_median
../../data/analysis/temporal_analysis/pivot_tables/accreditation-subject_matter/heatmap_bivar__accreditation-subject_matter-msg_count_facebook_median.pdf
plot_bivar_heatmap: accreditation-subject_matter msg_count_facebook_mean
../../data/analysis/temporal_analysis/pivot_tables/accreditation-subject_matter/heatmap_bivar__accreditation-subject_matter-msg_count_facebook_mean.pdf
plot_bivar_heatmap: accreditation-subject_matter msg

<Figure size 432x288 with 0 Axes>

### All tweets/posts over time

Summarise tweets using pandas aggregation.

In [ ]:
def generate_time_plots(df, label, aggr_func):
    print('generate_time_plots', label)
    print('date range:', min(df.index), max(df.index))
    tdf = pd.DataFrame()
    for time_aggr in ['W','D','M','Q']:
        print(time_aggr)
        tmpdf = aggr_func(df, time_aggr)

        # plot tweets
        fig, ax = plt.subplots()
        tmpdf['count'].plot(linewidth=1, figsize=(20,7))
        plt.title('All museums - N={} - {} over time by {}'.format(len(df), label, time_aggr))
        plt.xlabel('Time')
        plt.grid(True, which='both')
        plt.ylabel('N  '+label)
        #ax.xaxis.set_minor_formatter(mdates.DateFormatter('%m'))
        fig_fn = out_folder+'data/analysis/temporal_analysis/{}_temporal-all_mus-{}.pdf'.format(label,time_aggr)
        plt.savefig(fig_fn)
        print(fig_fn)
        #plt.show()
        
        plt.clf(); plt.cla(); plt.close()

        # add data
        tmpdf = tmpdf.reset_index()
        tdf.append(tmpdf)

    return tdf

def count_messages(df, time_aggr):
    tmpdf = df.resample(time_aggr).size().to_frame('count')
    tmpdf['time_unit'] = time_aggr
    return tmpdf

# generate tweet plots
df = pd.read_pickle(tw_fn)
print(len(df))
print(df.sample(10))
generate_time_plots(df, 'tweets', count_messages)

# generate facebook plots
df = pd.read_pickle(fb_fn)
print(len(df))
print(df.sample(10))
generate_time_plots(df, 'facebook_posts', count_messages)

### Active museums over time

Count museums that are active on FB/TW in a give period

In [10]:
def count_museums(df, time_aggr):
    df = df[['museum_id']]
    tmpdf = df.resample(time_aggr).nunique()
    tmpdf.columns = ['count']
    tmpdf['time_unit'] = time_aggr
    return tmpdf

# generate tweet plots
df = pd.read_pickle(tw_fn)
print(len(df))
generate_time_plots(df, 'museums_on_tw', count_museums)

# generate facebook plots
df = pd.read_pickle(fb_fn)
print(len(df))
generate_time_plots(df, 'museums_on_fb', count_museums)

5653012
generate_time_plots museums_on_tw
date range: 2019-01-01 00:00:00+00:00 2021-10-15 12:45:39+00:00
W
../../data/analysis/temporal_analysis/museums_on_tw_temporal-all_mus-W.pdf
D
../../data/analysis/temporal_analysis/museums_on_tw_temporal-all_mus-D.pdf
M
../../data/analysis/temporal_analysis/museums_on_tw_temporal-all_mus-M.pdf
Q
../../data/analysis/temporal_analysis/museums_on_tw_temporal-all_mus-Q.pdf
1468515
generate_time_plots museums_on_fb
date range: 2019-01-01 00:00:00+00:00 2021-10-20 23:43:03+00:00
W
../../data/analysis/temporal_analysis/museums_on_fb_temporal-all_mus-W.pdf
D
../../data/analysis/temporal_analysis/museums_on_fb_temporal-all_mus-D.pdf
M
../../data/analysis/temporal_analysis/museums_on_fb_temporal-all_mus-M.pdf
Q
../../data/analysis/temporal_analysis/museums_on_fb_temporal-all_mus-Q.pdf


""


### Daily heatmaps

- Calplot package: https://www.analyticsvidhya.com/blog/2021/02/visualization-in-time-series-using-heatmaps-in-python/

In [14]:
!pip install calplot

In [51]:
import calplot

def plot_daily_heatmap(df, label):
    # count tweets per day
    cmap = None
    cdf = df[['museum_id']].resample('D').count()
    ax = calplot.calplot(cdf['museum_id'], yearlabels=True)
    fig_fn = out_folder+'data/analysis/temporal_analysis/daily_heatmap-all_mus_heatmap_{}-msg.pdf'.format(label)
    plt.savefig(fig_fn)
    plt.clf()
    del cdf
    # count active museums per day
    mdf = df[['museum_id']].resample('D').nunique()
    ax = calplot.calplot(mdf['museum_id'], yearlabels=True)
    fig_fn = out_folder+'data/analysis/temporal_analysis/daily_heatmap-all_mus_heatmap_{}-active_mus.pdf'.format(label)
    #plt.tight_layout()
    plt.savefig(fig_fn)
    print(fig_fn)
    plt.clf()
    del mdf

df = pd.read_pickle(tw_fn)
plot_daily_heatmap(df, 'twitter')

df = pd.read_pickle(fb_fn)
plot_daily_heatmap(df, 'facebook')

/Users/andreaballatore/opt/anaconda3/envs/mip_v1/lib/python3.8/site-packages/calplot/calplot.py:189: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  ax.set_xticks([by_day.loc[pd.Timestamp(
/Users/andreaballatore/opt/anaconda3/envs/mip_v1/lib/python3.8/site-packages/calplot/calplot.py:189: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  ax.set_xticks([by_day.loc[pd.Timestamp(
/Users/andreaballatore/opt/anaconda3/envs/mip_v1/lib/python3.8/site-packages/calplot/calplot.py:189: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  ax.set_xticks([by_day.loc[pd.Timestamp(
/Users/andreaballatore

../../data/analysis/temporal_analysis/daily_heatmap-all_mus_heatmap_twitter-active_mus.pdf


/Users/andreaballatore/opt/anaconda3/envs/mip_v1/lib/python3.8/site-packages/calplot/calplot.py:189: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  ax.set_xticks([by_day.loc[pd.Timestamp(
/Users/andreaballatore/opt/anaconda3/envs/mip_v1/lib/python3.8/site-packages/calplot/calplot.py:189: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  ax.set_xticks([by_day.loc[pd.Timestamp(
/Users/andreaballatore/opt/anaconda3/envs/mip_v1/lib/python3.8/site-packages/calplot/calplot.py:189: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  ax.set_xticks([by_day.loc[pd.Timestamp(
/Users/andreaballatore

../../data/analysis/temporal_analysis/daily_heatmap-all_mus_heatmap_facebook-active_mus.pdf


<Figure size 900x367.2 with 0 Axes>

<Figure size 900x367.2 with 0 Axes>

<Figure size 900x367.2 with 0 Axes>

<Figure size 900x367.2 with 0 Axes>

End of notebook.